# Anwendungsbeispiel: Stellenwerk

Benötigte Bibliotheken laden.

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time

Die URL vom Stellenwerk enthält query strings

In [2]:
base_url = "https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs"
query = {
    'keywords': '',
    'offer_type': 'job_ad_b2b',
    'job_category': '111',
    'employment_type': 'All',
    'page': '0'
}

In [3]:
r = requests.get(base_url, params = query)

In [4]:
print(r.request.url)

https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=0


---

In [5]:
print(r.status_code)
print(r.reason)

200
OK


In [6]:
html = r.text

In [7]:
bs = BeautifulSoup(html, 'html.parser')

Die Anzeigen sind über mehrere Seiten verteilt. Wir müssen also blättern. Wie oft geblättert werden muss, kann man der Seite entnehmen:

In [8]:
pages = bs.find('span', class_ = 'Pagination-text').get_text()
print(pages)

Seite 1 von 50


Aus diesem String kann man die maximale Seitenzahl folgendermaßen extrahieren. Der String wird über die Methode `.split()` am Leerzeichen (' ') in seine Komponenten zerlegt. Aus dieser Liste wird dann das vierte Element extrahiert. Das vierte Element hat den Index 3, da die Indexzählung bei 0 beginnt. Schließlich wird der String mit `int()` in ein Integer umgewandelt und ist somit numerisch:

In [9]:
page_max = int(pages.split(' ')[3])

In [10]:
print(page_max)
print(type(page_max))

50
<class 'int'>


Nun kann diese Info genutzt werden, um zu blättern:

In [13]:
for page in range(page_max + 1):
    query["page"] = page
    r = requests.get(base_url, params = query)
    time.sleep(2)
    print(r.request.url)  

0
https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=0
1
https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=1
2
https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=2
3
https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=3
4
https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=4
5


KeyboardInterrupt: 

In [75]:
def stellenwerk(url):
    html = r.text
    bs = BeautifulSoup(html, 'html.parser')
    job_ads = bs.find('div', class_ = 'JobSearch-content')
    job_links = job_ads.find_all('a', class_ = 'Link')
    rows = []
    for link in job_links:
        text = link.get_text()
        link = 'https://www.stellenwerk-hamburg.de' + link['href']
        row = {
        'text': text,
        'links': link
        }
    rows.append(row)

Extrahierte Daten (Text und Link) in eine CSV-Datei schreiben:

In [76]:
file = 'daten/job_ads.csv'
with open(file, mode = 'w', newline = '') as f:
    col_names = ['text', 'links']
    writer = csv.DictWriter(f, fieldnames = col_names)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

In [9]:
%cpaste

UsageError: Line magic function `%cpaste` not found.
